# Import stuff

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set(context='notebook', font_scale=1.5, style='white', rc={'axes.facecolor':'#fcfcfc', 'figure.facecolor':'#fcfcfc'})
from scipy import stats
import random, time, sys, os, json

# Set up genetic operators

In [ ]:
def tournament_selection(pop, group_size=5):
    s = len(pop)
    i = int(random.random() * s)
    for _ in range(1, group_size):
        j = int(random.random() * s)
        if pop[i][1] < pop[j][1]:
            i = j
        
    return i

def proportional_selection(pop, cum):
    c = random.random() * cum[-1]
    for i, p in enumerate(pop):
        if c < cum[i]:
            return i

def get_best(pop):
    f = lambda x: x[1][1]
    return max(enumerate(p for p in pop if p is not None), key=f)[0]


# Standard GP

In [ ]:
def run_gp(data_path, run_index, algorithm_name, population_size, max_evaluations, tournament_size, p_crossover, p_mutation):
    base_path = os.path.dirname(data_path)
    
    with open(data_path, 'r') as h:
        info           = json.load(h)
        metadata       = info['metadata']
        target         = metadata['target']
        training_rows  = metadata['training_rows']
        training_start = training_rows['start']
        training_end   = training_rows['end']
        test_rows      = metadata['test_rows']
        test_start     = test_rows['start']
        test_end       = test_rows['end']
        problem_name   = metadata['name']
        problem_csv    = metadata['filename']
        
        datafile       = os.path.join(base_path, problem_csv)
    
        import _operon as Operon # operon python bindings
        ds             = Operon.Dataset(datafile, has_header=True)
        training_range = Operon.Range(training_start, training_end)
        test_range     = Operon.Range(test_start, test_end)
        target         = ds.GetVariable('Y')
        inputs         = Operon.VariableCollection(v for v in ds.Variables if v.Name != target.Name)

        y_train        = ds.Values[training_start:training_end, target.Index]

        rng            = Operon.RomuTrio(random.randint(1, 1000000))
        # some more common parameters
        min_length, max_length = 1, 50
        min_depth, max_depth   = 1, 10
        p_internal = 0.9 # crossover
        
        pset           = Operon.PrimitiveSet()
        pset.SetConfig(Operon.PrimitiveSet.Arithmetic | Operon.NodeType.Exp | Operon.NodeType.Log | Operon.NodeType.Sin | Operon.NodeType.Cos)
        
        # tree creator
        initial_lengths = np.random.randint(min_length, max_length+1, population_size)
        btc             = Operon.BalancedTreeCreator(pset, inputs, bias=0.0)
        grow            = Operon.GrowTreeCreator(pset, inputs)

        # crossover
        crossover      = Operon.SubtreeCrossover(p_internal, max_depth, max_length)

        # mutation
        mut_onepoint   = Operon.OnePointMutation()
        mut_changeVar  = Operon.ChangeVariableMutation(inputs)
        mut_changeFunc = Operon.ChangeFunctionMutation(pset)
        mut_replace    = Operon.ReplaceSubtreeMutation(btc, max_depth, max_length)
        mutation       = [ mut_onepoint, mut_changeFunc, mut_replace, mut_changeVar ]

        # offspring generator (applies selection, crossover and mutation)
        # to create one offspring individual
        # this is the way a GP algorithm usually defines recombination
        def generate(pop):
            do_crossover = random.uniform(0, 1) < p_crossover
            do_mutation = random.uniform(0, 1) < p_mutation

            i1 = tournament_selection(pop, tournament_size)
            p1, f1 = pop[i1]

            if do_crossover:
                i2 = tournament_selection(pop, tournament_size)
                p2, f2 = pop[i2]
                child = crossover(rng, p1, p2)

            if do_mutation:
                op = random.choice(mutation)
                p1 = child if do_crossover else p1
                child = op(rng, child)

            if do_crossover or do_mutation:
                return child, Operon.CalculateFitness(child, ds, training_range, target.Name, metric='rsquared')
            else:
                return p1,f1

        t0 = time.time()
        trees = [btc(rng, l, 0, 0) for l in initial_lengths]
        fit = [Operon.CalculateFitness(t, ds, training_range, target.Name, metric='rsquared') for t in trees]
        pop = [p for p in zip(trees, fit)]
        best = get_best(pop)
        
        r2_test = Operon.CalculateFitness(pop[best][0], ds, test_range, target.Name, metric='rsquared')
        
        evaluations = population_size

        df = pd.DataFrame(columns=['Algorithm', 'Run index', 'Generation', 'Problem', 'R2 train', 'R2 test', 'Best quality', 'Avg quality', 'Avg length', 'Evaluated solutions'])
        
        row = [
            algorithm_name,
            run_index,
            0,
            problem_name,
            pop[best][1],
            r2_test,
            pop[best][1],
            np.mean([p[1] for p in pop]),
            np.mean([p[0].Length for p in pop]),
            evaluations
        ]
        df.loc[0] = row
        
        gen = 1
        while True:
            remaining_budget = min(population_size, max_evaluations - evaluations)
            
            if remaining_budget <= 0:
                break
            
            pop = [pop[best] if i == best else generate(pop) for i in range(remaining_budget)]
            new_best = get_best(pop)
            
            evaluations += remaining_budget 
         
            if new_best != best:
                best = new_best
                r2_test = Operon.CalculateFitness(pop[best][0], ds, test_range, target.Name, metric='rsquared')

            row = [
                algorithm_name,
                run_index,
                gen,
                problem_name,
                pop[best][1],
                r2_test,
                pop[best][1],
                np.mean([p[1] for p in pop]),
                np.mean([p[0].Length for p in pop]),
                evaluations
            ]
            df.loc[gen] = row
            
            gen += 1

        t1 = time.time()
        return df

    
 

# Offspring selection GP

In [ ]:
def run_osgp(data_path, run_index, algorithm_name, population_size, max_evaluations, max_selection_pressure, tournament_size, p_crossover, p_mutation):
    base_path = os.path.dirname(data_path)
    
    with open(data_path, 'r') as h:
        info           = json.load(h)
        metadata       = info['metadata']
        target         = metadata['target']
        training_rows  = metadata['training_rows']
        training_start = training_rows['start']
        training_end   = training_rows['end']
        test_rows      = metadata['test_rows']
        test_start     = test_rows['start']
        test_end       = test_rows['end']
        problem_name   = metadata['name']
        problem_csv    = metadata['filename']
        
        datafile       = os.path.join(base_path, problem_csv)
    
        import _operon as Operon # operon python bindings
        ds             = Operon.Dataset(datafile, has_header=True)
        training_range = Operon.Range(training_start, training_end)
        test_range     = Operon.Range(test_start, test_end)
        target         = ds.GetVariable('Y')
        inputs         = Operon.VariableCollection(v for v in ds.Variables if v.Name != target.Name)

        y_train        = ds.Values[training_start:training_end, target.Index]

        rng            = Operon.RomuTrio(random.randint(1, 1000000))
        # some more common parameters
        min_length, max_length = 1, 50
        min_depth, max_depth   = 1, 10
        p_internal = 0.9 # crossover
        
        pset        = Operon.PrimitiveSet()
        pset.SetConfig(Operon.PrimitiveSet.Arithmetic | Operon.NodeType.Exp | Operon.NodeType.Log | Operon.NodeType.Sin | Operon.NodeType.Cos)
        
        # tree creator
        initial_lengths = np.random.randint(min_length, max_length+1, population_size)
        btc             = Operon.BalancedTreeCreator(pset, inputs, bias=0.0)
        grow            = Operon.GrowTreeCreator(pset, inputs)

        # crossover
        crossover      = Operon.SubtreeCrossover(p_internal, max_depth, max_length)

        # mutation
        mut_onepoint   = Operon.OnePointMutation()
        mut_changeVar  = Operon.ChangeVariableMutation(inputs)
        mut_changeFunc = Operon.ChangeFunctionMutation(pset)
        mut_replace    = Operon.ReplaceSubtreeMutation(btc, max_depth, max_length)
        mutation       = [ mut_onepoint, mut_changeFunc, mut_replace, mut_changeVar ]

        generated = 0    
        cumsum = [0] * population_size

        def generate(pop, comparison_factor=0):
            nonlocal generated
            nonlocal cumsum
            
            if generated / population_size > max_selection_pressure:
                return None
            
            while True:
                generated += 1

                do_crossover = random.uniform(0, 1) < p_crossover
                do_mutation = random.uniform(0, 1) < p_mutation

                i1 = proportional_selection(pop, cumsum)
#                 i1 = tournament_selection(pop, 2)
#                 i1 = random.randrange(population_size)
                p1, f1 = pop[i1]
                f2 = 0

                if do_crossover:
                    i2 = random.randrange(population_size)
                    p2, f2 = pop[i2]
                    child = crossover(rng, p1, p2)

                if do_mutation:
                    op = random.choice(mutation)
                    p1 = child if do_crossover else p1
                    child = op(rng, child)

                if do_crossover or do_mutation:
                    fc = Operon.CalculateFitness(child, ds, training_range, target.Name, metric='rsquared')

                    if fc > min(f1, f2) + comparison_factor * abs(f1-f2):
                        return child, fc

                if not generated / population_size < max_selection_pressure:
                    break

            return None
    
        t0 = time.time()
        trees = [btc(rng, l, 0, 0) for l in initial_lengths]
        fit = [Operon.CalculateFitness(t, ds, training_range, target.Name, metric='rsquared') for t in trees]
        pop = [p for p in zip(trees, fit)]
        best = get_best(pop)
        r2_test = Operon.CalculateFitness(pop[best][0], ds, test_range, target.Name, metric='rsquared')
        
        evaluations = population_size

        df = pd.DataFrame(columns=['Algorithm', 'Run index', 'Generation', 'Problem', 'R2 train', 'R2 test', 'Best quality', 'Avg quality', 'Avg length', 'Evaluated solutions'])
        
        row = [
            algorithm_name,
            run_index,
            0,
            problem_name,
            pop[best][1],
            r2_test,
            pop[best][1],
            np.mean([p[1] for p in pop]),
            np.mean([p[0].Length for p in pop]),
            evaluations
        ]
        df.loc[0] = row

        gen = 1
        while True:
            remaining_budget = min(population_size, max_evaluations - evaluations)
            
            if remaining_budget <= 0:
                break
            
            # precalculate the cummulative fitness sum, for proportional selection
            c = 0
            for i, p in enumerate(pop):
                c += pop[i][1]
                cumsum[i] = c
                
            pop = [pop[best] if i == best else generate(pop, 1.0) for i in range(remaining_budget)]
            new_best = get_best(pop)
            
            selection_pressure = generated / population_size
            evaluations += generated
            generated = 0
            
            if new_best is None:
                break

            if new_best != best:
                best = new_best
                r2_test = Operon.CalculateFitness(pop[best][0], ds, test_range, target.Name, metric='rsquared')

            row = [
                algorithm_name,
                run_index,
                gen,
                problem_name,
                pop[best][1],
                r2_test,
                pop[best][1],
                np.mean([p[1] for p in pop if p is not None]),
                np.mean([p[0].Length for p in pop if p is not None]),
                evaluations
            ]
            df.loc[gen] = row
            gen += 1
            
            if selection_pressure > max_selection_pressure or evaluations > max_evaluations:
                break

        t1 = time.time()
        return df
    

# Run GP

In [ ]:
if __name__ == '__main__':
    from multiprocessing import Pool, Manager
    manager = Manager()
    results = manager.list()
    
    def runner_gp(i):
        res = run_gp(data_path='../data/Poly-10.json', run_index=i, algorithm_name='GP', population_size=1000, max_evaluations=1000 * 1000, 
                     tournament_size=5, p_crossover=1.0, p_mutation=0.25)
        results.append(res)
        
    def runner_osgp(i):
        res = run_osgp(data_path='../data/Poly-10.json', run_index=i, algorithm_name='OSGP', population_size=1000, max_evaluations=1000 * 1000, max_selection_pressure=100, 
                     tournament_size=5, p_crossover=1.0, p_mutation=0.25)
        results.append(res)
    
    pool = Pool()
    reps = 10
    pool.map(runner_gp, range(reps))
    print('GP done')
    pool.map(runner_osgp, range(reps))
    print('OSGP done')
    df = pd.concat(results)
    
#     print(results)


# Visualize the results

In [ ]:
    import seaborn as sns
    from matplotlib import rc
    import matplotlib.pyplot as plt
    sns.set(context='notebook', font_scale=2, style='white')
    rc('text', usetex=True)
    rc('text.latex', preamble=r'\usepackage{lmodern}')
    from scipy.signal import savgol_filter
    
    print(df.groupby(['Algorithm']).mean())
    print(df.groupby(['Algorithm']).median())
    
    df_gp = df[df['Algorithm'] == 'GP']
    df_osgp = df[df['Algorithm'] == 'OSGP']
#     print(df_osgp.head)
    intervals = np.zeros(10)
    df_last = df_osgp.groupby(['Run index']).last()
    for v in df_last['R2 train']:
        i = int(v * 10)
        for j in range(i+1):
            intervals[j] += 1
    print(intervals)
    

In [ ]:
    def round_down(v, p=3):
        n = 10 ** p
        return v // n * n
        
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,4))    
    sns.lineplot(data=df_gp, x='Evaluated solutions', y='R2 train', ax=ax[0], ci=None)
    ax[0].set(xlabel='Evaluated solutions', ylim=(0,1))
    sns.lineplot(data=df_gp, x='Evaluated solutions', y='R2 test', ax=ax[1], ci=None)
    ax[1].set(xlabel='Evaluated solutions', ylim=(0,1))
    
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,4))
    sns.lineplot(data=df_gp, x='Evaluated solutions', y='Avg quality', ax=ax[0], ci=None)
    sns.lineplot(data=df_gp, x='Evaluated solutions', y='Avg length', ax=ax[1], ci=None)
    ax[1].set(xlabel='Evaluated solutions', ylim=(0, 50))
    ax[1].set_yticks(np.arange(0, 50, 10))
    # for OSGP we want to do some smoothing of the values
#     filtered =  savgol_filter(df['R2 train'], 31, 3)
    
    df_osgp.loc[:, ('Evaluated solutions rounded')] = df_osgp['Evaluated solutions'].apply(lambda x: round_down(x, 4))
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,4))    
    sns.lineplot(data=df_osgp, x='Evaluated solutions rounded', y='R2 train', ax=ax[0], ci=None)
    ax[0].set(xlabel='Evaluated solutions')
    sns.lineplot(data=df_osgp, x='Evaluated solutions rounded', y='R2 test', ax=ax[1], ci=None)
    ax[1].set(xlabel='Evaluated solutions')
    
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,4))
    sns.lineplot(data=df_osgp, x='Evaluated solutions rounded', y='Avg quality', ax=ax[0], ci=None)
    sns.lineplot(data=df_osgp, x='Evaluated solutions rounded', y='Avg length', ax=ax[1], ci=None)
    ax[1].set(xlabel='Evaluated solutions', ylim=(0, 50))
    ax[1].set_yticks(np.arange(0, 50, 10))
#     fig.tight_layout()